In [1]:
from datetime import datetime, timedelta
from typing import Iterable, List, Mapping, Union

In [2]:
class SliceGrid:
    
    NMAX = 62
    GRID_DURATION = 95.01580774
    START_OVERLAP = 5
    END_OVERLAP = 7
    THRESHOLD = 10
    
    def __init__(self, anx_times: Iterable[datetime]):
        self._anx_times = sorted(anx_times)
        if len(self._anx_times) > 1:
            for n in range(len(self._anx_times) - 1):
                delta_t = abs(
                    (self._anx_times[n + 1] -
                     self._anx_times[n] -
                     timedelta(seconds=self.NMAX * self.GRID_DURATION)).total_seconds()
                )
                if delta_t > self.THRESHOLD:
                    raise ValueError(
                        f'abs(anx_time[{n + 1}] - anx_time[{n}]) = {delta_t} > {self.THRESHOLD}'
                    )
    
    def tie_points(self) \
            -> List[Mapping[str, Union[int, float, datetime]]]:
        result = [
            {
                'number': n + 1,
                'anx_time': anx_time,
                'grid_start': anx_time + timedelta(seconds=n * self.GRID_DURATION),
                'grid_stop': anx_time + timedelta(seconds=(n + 1) * self.GRID_DURATION),
                'offset': 0
            }
            for anx_time in self._anx_times
            for n in range(self.NMAX)
        ]
        for n in range(len(result) - 1):
            if result[n]['number'] == self.NMAX:
                result[n]['grid_stop'] = result[n + 1]['anx_time']
                result[n]['offset'] = (
                    result[n + 1]['anx_time'] -
                    (result[n]['anx_time'] + timedelta(seconds=self.NMAX * self.GRID_DURATION))
                ).total_seconds()
        return result
    
    def slice_range(self, start_date: datetime, stop_date: datetime) \
            -> List[Mapping[str, Union[int, float, datetime]]]:
        result = [
            record for record in self.tie_points()
            if record['grid_start'] <= stop_date and record['grid_stop'] >= start_date
        ]
        for n in range(len(result)):
            result[n]['slice_start'] = result[n]['grid_start'] - timedelta(seconds=self.START_OVERLAP)
            result[n]['slice_stop'] = result[n]['grid_stop'] + timedelta(seconds=self.END_OVERLAP)
        return result
    

In [3]:
ORBIT_DURATION = 86400 * 3 / 44

anx_times = [datetime(2023, 9, 1) + timedelta(seconds=n * ORBIT_DURATION) for n in range(2)]

In [4]:
anx_times

[datetime.datetime(2023, 9, 1, 0, 0),
 datetime.datetime(2023, 9, 1, 1, 38, 10, 909091)]

In [5]:
grid = SliceGrid(anx_times)

In [6]:
grid.slice_range(datetime(2023, 9, 1, 1, 36), datetime(2023, 9, 1, 1, 40))

[{'number': 61,
  'anx_time': datetime.datetime(2023, 9, 1, 0, 0),
  'grid_start': datetime.datetime(2023, 9, 1, 1, 35, 0, 948464),
  'grid_stop': datetime.datetime(2023, 9, 1, 1, 36, 35, 964272),
  'offset': 0,
  'slice_start': datetime.datetime(2023, 9, 1, 1, 34, 55, 948464),
  'slice_stop': datetime.datetime(2023, 9, 1, 1, 36, 42, 964272)},
 {'number': 62,
  'anx_time': datetime.datetime(2023, 9, 1, 0, 0),
  'grid_start': datetime.datetime(2023, 9, 1, 1, 36, 35, 964272),
  'grid_stop': datetime.datetime(2023, 9, 1, 1, 38, 10, 909091),
  'offset': -0.070989,
  'slice_start': datetime.datetime(2023, 9, 1, 1, 36, 30, 964272),
  'slice_stop': datetime.datetime(2023, 9, 1, 1, 38, 17, 909091)},
 {'number': 1,
  'anx_time': datetime.datetime(2023, 9, 1, 1, 38, 10, 909091),
  'grid_start': datetime.datetime(2023, 9, 1, 1, 38, 10, 909091),
  'grid_stop': datetime.datetime(2023, 9, 1, 1, 39, 45, 924899),
  'offset': 0,
  'slice_start': datetime.datetime(2023, 9, 1, 1, 38, 5, 909091),
  'slice